# AoC - Day 3

## Part 1 - Problem Statement

Opening the front panel reveals a jumble of wires. Specifically, two wires are connected to a central port and extend outward on a grid. You trace the path each wire takes as it leaves the central port, one wire per line of text (your puzzle input).

The wires twist and turn, but the two wires occasionally cross paths. To fix the circuit, you need to find the intersection point closest to the central port. Because the wires are on a grid, use the Manhattan distance for this measurement. While the wires do technically cross right at the central port where they both start, this point does not count, nor does a wire count as crossing with itself.

## Solution

In [6]:
from operator import itemgetter

In [9]:
def parse_instruction_string(instruction):
    return {'direction': instruction[0],
            'steps': int(instruction[1:])}

In [10]:
def parse_instructions_string(instructions):
    return [parse_instruction_string(instruction)
            for instruction in instructions.split(',')]

In [52]:
def extend_path(current_location, instruction_dict):
    direction, steps = itemgetter('direction', 'steps')(instruction_dict)
    x, y = current_location
    
    if direction == 'U':
        return [(x, s) for s in range(y + 1, y + steps + 1)]
    if direction == 'D':
        return [(x, s) for s in range(y - 1, y - steps - 1, -1)]
    if direction == 'R':
        return [(s, y) for s in range(x + 1, x + steps + 1)]
    if direction == 'L':
        return [(s, y) for s in range(x - 1, x - steps - 1, -1)]
    
    assert False

In [65]:
def build_path(instructions):
    instructions = parse_instructions_string(instructions)
    
    path = [(0, 0)]
    for instruction in instructions:
        path.extend(extend_path(path[-1], instruction))
    
    return path

In [71]:
def build_paths(instructions_list):
    return [build_path(instructions) for instructions in instructions_list]

In [76]:
def find_intersections(instructions_list):
    paths = build_paths(instructions_list)
    path_coords = [set(path) for path in paths]
    
    return set.intersection(*path_coords) - {(0, 0)}

In [83]:
def compute_distance(coords):
    return abs(coords[0]) + abs(coords[1])

In [89]:
def compute_distances(coords):
    return [{'coords': x_y, 'distance': compute_distance(x_y)} for x_y in coords]

In [92]:
def find_closest_intersection(instructions_list):
    intersections = find_intersections(instructions_list)
    distances = compute_distances(intersections)
    
    return min(distances, key=lambda x: x['distance'])

## Tests

In [12]:
assert parse_instruction_string('R10') == {'direction': 'R', 'steps': 10}
assert parse_instructions_string('R10,U5') == [
    {'direction': 'R', 'steps':10},
    {'direction': 'U', 'steps':5}
]

In [51]:
assert extend_path((0, 0), parse_instruction_string('U3')) == [(0, 1), (0, 2), (0, 3)]
assert extend_path((0, 0), parse_instruction_string('D3')) == [(0, -1), (0, -2), (0, -3)]
assert extend_path((0, 0), parse_instruction_string('R3')) == [(1, 0), (2, 0), (3, 0)]
assert extend_path((0, 0), parse_instruction_string('L3')) == [(-1, 0), (-2, 0), (-3, 0)]

In [66]:
assert build_path('U1,R1') == [(0, 0), (0, 1), (1, 1)]
assert build_path('R1,U1') == [(0, 0), (1, 0), (1, 1)]
assert build_path('D1,L1') == [(0, 0), (0, -1), (-1, -1)]
assert build_path('L1,D1') == [(0, 0), (-1, 0), (-1, -1)]

In [72]:
assert build_paths(['U1,R1', 'R1,U1']) == [[(0, 0), (0, 1), (1, 1)],
                                           [(0, 0), (1, 0), (1, 1)]]

In [80]:
assert find_intersections(['U1,R1', 'R1,U1']) == {(1, 1)}

In [82]:
assert compute_distance((2, 3)) == 5
assert compute_distance((-2, -3)) == 5

In [88]:
assert compute_distances([(1, 1), (-2, 2)]) == [{'coords': (1, 1), 'distance': 2},
                                                {'coords': (-2, 2), 'distance': 4}]

In [95]:
assert find_closest_intersection(['R75,D30,R83,U83,L12,D49,R71,U7,L72',
                                  'U62,R66,U55,R34,D71,R55,D58,R83'])['distance'] == 159

assert find_closest_intersection(['R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51',
                                  'U98,R91,D20,R16,D67,R40,U7,R15,U6,R7'])['distance'] == 135

## Input

In [96]:
data = ['R999,U626,R854,D200,R696,D464,R54,D246,L359,U57,R994,D813,L889,U238,L165,U970,L773,D904,L693,U512,R126,D421,R732,D441,R453,D349,R874,D931,R103,D794,R934,U326,L433,D593,L984,U376,R947,U479,R533,U418,R117,D395,L553,D647,R931,D665,L176,U591,L346,D199,L855,D324,L474,U251,R492,D567,L97,D936,L683,U192,R198,U706,L339,U66,R726,D102,R274,U351,R653,D602,L695,U921,R890,D654,R981,U351,R15,U672,R856,D319,R102,D234,R248,U169,L863,U375,L412,U75,L511,U298,L303,U448,R445,U638,L351,D312,R768,D303,L999,D409,L746,U266,L16,U415,L951,D763,L976,U342,L505,U770,L228,D396,L992,U3,R243,D794,L496,U611,R587,U772,L306,D119,L470,D490,L336,U518,L734,D654,L150,U581,L874,U691,L243,U94,L9,D582,L402,U563,R468,U96,L311,D10,R232,U762,R630,D1,L674,U685,R240,D907,R394,U703,L64,U397,L810,D272,L996,D954,R797,U789,R790,D526,R103,D367,R143,D41,L539,D735,R51,D172,L33,U241,R814,D981,R748,D699,L716,U647,L381,D351,L381,D121,L52,U601,R515,U713,L404,U45,R362,U670,L235,U102,R373,U966,L362,U218,R280,U951,R371,U378,L10,U670,R958,D423,L740,U888,R235,U899,L387,U167,R392,D19,L330,D916,R766,D471,L708,D83,R749,D696,L50,D159,R828,U479,L980,D613,L182,D875,L307,U472,L317,U999,R435,D364,R737,U550,L233,U190,L501,U610,R433,U470,L801,U52,L393,D596,L378,U220,L967,D807,R357,D179,L731,D54,L804,D865,L994,D151,L181,U239,R794,D378,L487,U408,R817,U809,R678,D599,L564,U480,R525,D189,L641,D771,L514,U72,L248,D334,L859,D318,R590,D571,R453,U732,R911,U632,R992,D80,R490,D234,L710,U816,L585,U180,L399,D238,L103,U605,R993,D539,R330',
        'L996,U383,L962,U100,L836,D913,R621,U739,R976,D397,L262,D151,L12,U341,R970,U123,L713,U730,L52,D223,L190,D81,R484,D777,R374,U755,R640,D522,R603,D815,R647,U279,R810,U942,R314,D19,L938,U335,R890,U578,R273,U338,R186,D271,L230,U90,R512,U672,R666,D328,L970,U17,R368,D302,L678,D508,L481,U12,L783,D409,L315,D579,L517,D729,R961,D602,R253,D746,R418,D972,R195,D270,L46,D128,L124,U875,R632,D788,L576,U695,R159,U704,R599,D597,R28,D703,L18,D879,L417,U633,L56,U302,R289,U916,R820,D55,R213,U712,R250,D265,L935,D171,L680,U738,L361,D939,R547,D606,L255,U880,R968,U255,R902,D624,L251,U452,L412,D60,L996,D140,L971,U196,R796,D761,L54,U54,L98,D758,L521,U578,L861,U365,L901,D495,L234,D124,L121,D329,L38,U481,L491,D938,L840,D311,L993,D954,R654,U925,L528,D891,L994,D681,L879,D476,L933,U515,L292,U626,R348,D963,L145,U230,L114,D11,R651,D929,R318,D672,R125,D827,L590,U338,L755,D925,L577,D52,R131,D465,R657,D288,R22,D363,R162,D545,L904,D457,R987,D389,L566,D931,L773,D53,R162,U271,L475,U666,L594,U733,R279,D847,R359,U320,R450,D704,L698,D173,R35,D267,L165,D66,L301,U879,R862,U991,R613,D489,L326,D393,R915,U718,R667,U998,R554,U199,R300,U693,R753,U938,R444,U12,L844,D912,R297,D668,R366,U710,L821,U384,R609,D493,R233,U898,R407,U683,R122,U790,L1,U834,L76,U572,R220,U752,L728,D85,L306,D805,R282,U507,R414,D687,L577,U174,L211,U308,L15,U483,R741,D828,L588,D192,L409,D605,L931,U260,L239,D424,L846,U429,L632,U122,L266,D544,R248,U188,R465,U721,R621,U3,L884,U361,L322,U504,R999,U381,R327,U555,L467,D849,R748,U175,R356']

In [97]:
find_closest_intersection(data)

{'coords': (399, 0), 'distance': 399}

## Part 2 - Problem Statement

It turns out that this circuit is very timing-sensitive; you actually need to minimize the signal delay.

To do this, calculate the number of steps each wire takes to reach each intersection; choose the intersection where the sum of both wires' steps is lowest. If a wire visits a position on the grid multiple times, use the steps value from the first time it visits that position when calculating the total value of a specific intersection.

The number of steps a wire takes is the total number of grid squares the wire has entered to get to that location, including the intersection being considered.

## Solution

In [108]:
def path_steps(path):
    steps = [(coords, step_count) for step_count, coords in enumerate(path, 0)]
    
    min_steps = {}
    for coords, step_count in steps:
        if not coords in min_steps:
            min_steps[coords] = step_count
            
    return min_steps

In [118]:
def find_step_counts(instructions_list):
    paths = build_paths(instructions_list)
    
    return [path_steps(path) for path in paths]

In [120]:
def find_closest_intersection_steps(instructions_list):
    
    step_counts = find_step_counts(instructions_list)
    intersections = find_intersections(instructions_list)
    
    distances = [sum(step_count[coords] for step_count in step_counts) for coords in intersections]
    
    return min(distances)

## Tests

In [121]:
assert path_steps(build_path('U1,R1')) == {(0, 0): 0, (0, 1): 1, (1, 1): 2}
assert path_steps(build_path('U1,R1,L1')) == {(0, 0): 0, (0, 1): 1, (1, 1): 2}

In [122]:
assert find_closest_intersection_steps(['R75,D30,R83,U83,L12,D49,R71,U7,L72',
                                        'U62,R66,U55,R34,D71,R55,D58,R83']) == 610
assert find_closest_intersection_steps(['R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51',
                                        'U98,R91,D20,R16,D67,R40,U7,R15,U6,R7']) == 410

In [123]:
find_closest_intersection_steps(data)

15678